<a href="https://colab.research.google.com/github/ApahSaroj/WPI/blob/main/WPI_EvidentlyAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/evidentlyai/evidently

In [1]:
pip install evidently

In [2]:
import pandas as pd
import numpy as np


from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_excel('/content/drive/MyDrive/DATA/WQC_test.xlsx')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.drop('WQC', inplace=True, axis=1)

pho = df['Phosphate']
pho

for i in range(len(pho)):
    if i>5:
        if np.isnan(pho[i]):
                pho[i] = np.mean([pho[i-1], pho[i-2], pho[i-3], pho[i-4], pho[i-5]])
pho

for i in range(4,-1,-1):
    if np.isnan(pho[i]):
            pho[i] = np.mean([pho[i+1], pho[i+2], pho[i+3], pho[i+4], pho[i+5]])
df

,pH,DO,TDS,Alkalinity,EC,Na,Ca,Mg,K,F,Cl,Nitrate,Sulphate,Phosphate,WPI
0,8.14,8.7,84.0,52.0,147.8,4.48,44.0,20.0,1.17,0.285,7.80,0.758000,6.534660,0.06211,0.249528
1,7.70,8.8,110.0,76.0,188.0,8.61,44.0,36.0,3.46,0.401,14.80,0.196062,8.138620,0.00960,0.216400
2,7.60,7.5,75.0,44.0,128.0,8.11,28.0,20.0,2.80,0.379,14.80,0.298875,3.855625,0.01280,0.183814
3,8.20,9.1,76.0,56.0,127.0,6.30,44.0,18.0,1.65,0.396,9.60,0.299590,2.238750,0.01536,0.219282
4,7.31,7.3,84.0,52.0,140.0,5.50,28.0,20.0,1.61,0.265,8.30,0.106000,5.600000,0.01000,0.165831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,7.60,5.6,224.0,8.0,370.0,16.14,92.0,24.0,7.60,2.270,23.04,1.352000,104.480000,0.01500,0.419630
483,7.30,5.6,266.0,96.0,434.0,20.56,96.0,32.0,9.89,1.550,30.86,7.074000,61.068000,0.03300,0.432996
484,5.10,5.8,290.0,12.0,487.0,37.35,60.0,44.0,14.80,0.159,60.13,9.256000,89.910000,0.16300,0.326811
485,7.40,6.4,230.0,72.0,385.0,20.06,70.0,42.0,10.35,1.360,32.92,7.110435,48.500000,0.21900,0.509970


In [4]:
from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab, NumTargetDriftTab

from evidently.model_profile import Profile
from evidently.profile_sections import DataDriftProfileSection, NumTargetDriftProfileSection

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [5]:
wpi_frame = df

In [6]:
wpi_frame.head()

,pH,DO,TDS,Alkalinity,EC,Na,Ca,Mg,K,F,Cl,Nitrate,Sulphate,Phosphate,WPI
0,8.14,8.7,84.0,52.0,147.8,4.48,44.0,20.0,1.17,0.285,7.8,0.758000,6.534660,0.06211,0.249528
1,7.70,8.8,110.0,76.0,188.0,8.61,44.0,36.0,3.46,0.401,14.8,0.196062,8.138620,0.00960,0.216400
2,7.60,7.5,75.0,44.0,128.0,8.11,28.0,20.0,2.80,0.379,14.8,0.298875,3.855625,0.01280,0.183814
3,8.20,9.1,76.0,56.0,127.0,6.30,44.0,18.0,1.65,0.396,9.6,0.299590,2.238750,0.01536,0.219282
4,7.31,7.3,84.0,52.0,140.0,5.50,28.0,20.0,1.61,0.265,8.3,0.106000,5.600000,0.01000,0.165831


In [7]:
wpi_data_drift_dashboard = Dashboard(tabs=[DataDriftTab])
wpi_data_drift_dashboard.calculate(wpi_frame[:200], wpi_frame[200:])

In [8]:
wpi_data_drift_dashboard.show()

In [9]:
wpi_data_drift_dashboard.save('reports/wpi_data_drift_no_mapping.html')

In [10]:
wpi_data_drift_profile = Profile(sections=[DataDriftProfileSection])

In [11]:
column_mapping = {}

column_mapping['target'] = 'target'
column_mapping['prediction'] = None
column_mapping['datetime'] = None

column_mapping['numerical_features'] = ['pH',	'DO',	'TDS',	'Alkalinity',	'EC',	'Na',	'Ca',	'Mg',	'K',	'F',	'Cl',	'Nitrate',	'Sulphate',	'Phosphate',	'WPI']


In [12]:
wpi_data_drift_profile.calculate(wpi_frame[:200], wpi_frame[200:], 
                                               column_mapping = column_mapping)

In [13]:
wpi_data_drift_profile.json()

'{"data_drift": {"name": "data_drift", "datetime": "2021-07-14 16:10:17.309611", "data": {"utility_columns": {"date": null, "id": null, "target": "target", "prediction": null}, "cat_feature_names": [], "num_feature_names": ["pH", "DO", "TDS", "Alkalinity", "EC", "Na", "Ca", "Mg", "K", "F", "Cl", "Nitrate", "Sulphate", "Phosphate", "WPI"], "metrics": {"pH": {"current_small_hist": [[0.010041269618130504, 0.0, 0.010041269618130504, 0.0, 0.010041269618130504, 0.040165078472522014, 0.3916095151070907, 0.7129301428872659, 1.2852825111207045, 0.4217333239614822], [5.1, 5.447, 5.794, 6.141, 6.4879999999999995, 6.835, 7.182, 7.529, 7.876, 8.223, 8.57]], "ref_small_hist": [[0.023809523809523815, 0.14285714285714288, 0.19047619047619052, 0.2619047619047619, 0.6904761904761907, 0.9285714285714287, 0.8333333333333335, 0.5714285714285715, 0.7380952380952414, 0.3809523809523794], [6.4, 6.61, 6.82, 7.03, 7.24, 7.45, 7.66, 7.87, 8.08, 8.29, 8.5]], "feature_type": "num", "p_value": 2.4885518790096484e-0

In [14]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

from evidently.dashboard import Dashboard
from evidently.tabs import RegressionPerformanceTab

from evidently.model_profile import Profile
from evidently.profile_sections import RegressionPerformanceProfileSection

In [15]:
ref_data = df[:387]
prod_data = df[387:487]

In [16]:
ref_data

,pH,DO,TDS,Alkalinity,EC,Na,Ca,Mg,K,F,Cl,Nitrate,Sulphate,Phosphate,WPI
0,8.14,8.7,84.0,52.0,147.8,4.48,44.0,20.0,1.17,0.285,7.8,0.758000,6.534660,0.06211,0.249528
1,7.70,8.8,110.0,76.0,188.0,8.61,44.0,36.0,3.46,0.401,14.8,0.196062,8.138620,0.00960,0.216400
2,7.60,7.5,75.0,44.0,128.0,8.11,28.0,20.0,2.80,0.379,14.8,0.298875,3.855625,0.01280,0.183814
3,8.20,9.1,76.0,56.0,127.0,6.30,44.0,18.0,1.65,0.396,9.6,0.299590,2.238750,0.01536,0.219282
4,7.31,7.3,84.0,52.0,140.0,5.50,28.0,20.0,1.61,0.265,8.3,0.106000,5.600000,0.01000,0.165831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,8.39,7.1,139.0,80.0,216.1,9.60,48.0,32.0,3.40,0.399,15.2,0.368150,10.693000,0.01600,0.279700
383,7.13,7.5,141.0,80.0,217.4,13.38,72.0,16.0,4.57,0.416,22.2,0.251055,15.247000,0.04090,0.263879
384,7.57,7.8,88.0,44.0,135.0,5.76,32.0,20.0,1.53,0.481,10.2,2.277029,17.720000,0.01300,0.195486
385,8.30,8.4,102.0,48.0,171.0,4.66,48.0,32.0,3.11,0.257,7.6,1.532392,18.810000,0.02900,0.247548


In [17]:
target = 'WPI'
datetime = None

numerical_features = ['pH',	'DO',	'TDS',	'Alkalinity',	'EC',	'Na',	'Ca',	'Mg',	'K',	'F',	'Cl',	'Nitrate',	'Sulphate',	'Phosphate']

features = numerical_features

In [18]:
model = RandomForestRegressor(random_state = 0)


In [19]:
model.fit(ref_data[features], ref_data[target])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [20]:

ref_data['prediction']  = model.predict(ref_data[features])
prod_data['prediction'] = model.predict(prod_data[features])

In [21]:
column_mapping = {}

column_mapping['target'] = target
column_mapping['prediction'] = 'prediction'
column_mapping['datetime'] = None

column_mapping['numerical_features'] = numerical_features


In [22]:
dashboard = Dashboard(tabs=[RegressionPerformanceTab])

In [23]:
dashboard .calculate(ref_data, prod_data, column_mapping=column_mapping)

In [24]:
dashboard.show()

In [25]:
dashboard.save('wpi_model_perfomance.html')

In [26]:
wpi_regression_performance_profile = Profile(sections=[RegressionPerformanceProfileSection])

In [27]:
wpi_regression_performance_profile.calculate(ref_data, prod_data, column_mapping=column_mapping)

In [28]:
regression_profile = wpi_regression_performance_profile.json()

In [29]:
regression_profile

'{"regression_performance": {"name": "regression_performance", "datetime": "2021-07-14 16:10:31.741365", "data": {"utility_columns": {"date": null, "id": null, "target": "WPI", "prediction": "prediction"}, "cat_feature_names": [], "num_feature_names": ["pH", "DO", "TDS", "Alkalinity", "EC", "Na", "Ca", "Mg", "K", "F", "Cl", "Nitrate", "Sulphate", "Phosphate"], "metrics": {"reference": {"mean_error": -0.001170745778754947, "mean_abs_error": 0.006181753568685647, "mean_abs_perc_error": 1.894188350415684, "error_std": 0.018136444319380437, "abs_error_std": 0.017087765557265915, "abs_perc_error_std": 2.2940749799149174, "error_normality": {"order_statistic_medians": [-0.3150307477984129, -0.061401070801586854, -0.03518781866944509, -0.03073089277974339, -0.028752697910317926, -0.02751230879365063, -0.026322401573015575, -0.023085617360317667, -0.02244820544960313, -0.02191740932790992, -0.019782476515872616, -0.0189350302623012, -0.01772463084126996, -0.01720330092777783, -0.01694543406706